# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**¡Empecemos!**
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [16]:
from selenium import webdriver
#importampos libreria para cargar el driver automaticamente
from webdriver_manager.firefox import GeckoDriverManager

url='https://www.latamairlines.com/ar/es/ofertas-vuelos?origin=ASU&inbound=null&outbound=2022-12-01T15%3A00%3A00.000Z&destination=MAD&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED'

Necesitamos controladores web para diferentes navegadores web.<br>
Paso 1: instanciar un **driver** del navegador

In [17]:
options = webdriver.FirefoxOptions()
# Podemos agregarle opciones al driver para utilizar los distintos modos del navegador
options.add_argument('-private')
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)

[WDM] - Downloading: 19.0kB [00:00, 6.52MB/s]                   
C:\Users\mario\AppData\Local\Temp\ipykernel_14588\1250048064.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)


Paso 2: hacer que el navegador cargue la página web.

In [18]:
driver.get(url)

paso 3:Extraer informacion de la pagina<br>
Carguemos la página y analicemos dónde se encuentra la información<br>
Vemos que el bloque de vuelos se encuentra en una `ul` y que cada vuelo es un item de la lista, `li`. <br>

In [19]:
#Usaremos el Xpath para obtener la lista de vuelos
vuelos = driver.find_elements('xpath','//ol/li')
print (vuelos)

[<selenium.webdriver.remote.webelement.WebElement (session="30b5a596-d633-47be-b92e-51501d732334", element="287fc6c8-855f-490a-8846-80bf147d27c1")>, <selenium.webdriver.remote.webelement.WebElement (session="30b5a596-d633-47be-b92e-51501d732334", element="59ed8c6b-287d-4fad-abe9-8f2e23972516")>, <selenium.webdriver.remote.webelement.WebElement (session="30b5a596-d633-47be-b92e-51501d732334", element="f5a980ec-4369-48b7-b5b3-e4ac8cd1ba5b")>, <selenium.webdriver.remote.webelement.WebElement (session="30b5a596-d633-47be-b92e-51501d732334", element="5f98a230-f7a1-4258-ad78-d80b9863515a")>, <selenium.webdriver.remote.webelement.WebElement (session="30b5a596-d633-47be-b92e-51501d732334", element="54562446-c699-4c0a-8e27-3f6771285e5b")>, <selenium.webdriver.remote.webelement.WebElement (session="30b5a596-d633-47be-b92e-51501d732334", element="8e0a6367-28f8-4524-a09c-24ee96cf1c98")>, <selenium.webdriver.remote.webelement.WebElement (session="30b5a596-d633-47be-b92e-51501d732334", element="88ee

Obtengamos la informacion de la hora de salida, llegada y duracion del vuelo

Notar que el xpath comienza con ".", lo que indica que sólo debe buscar en los hijos de ese elemento. Si no ponemos el ".", busca en todo el árbol.<br>
Elementos importantes de xpath:
- "//" busca en todos los hijos del elemento. "/" busca sólo en hijos directos
- "." indica que la búsqueda debe empezar en ese elemento y no en el origen del árbol
- Los atributos de los tags se buscan entre [] y con @

In [20]:
#seleccionamos el primer vuelo
vuelo_1=vuelos[0]
#hora de salida
hora_salida=vuelo_1.find_element('xpath','//div[@class="sc-ixltIz dfdfxH flight-information"]/span[1]').text
print (hora_salida)

10:50


In [21]:
#hora de llegada
hora_llegada=vuelo_1.find_element('xpath','.//div[3]/span[1]').text.replace('\n+1','')
print (hora_llegada)

13:55


In [22]:
# Duracion del vuelo
duracion_vuelo= vuelo_1.find_element('xpath','.//div[2]/span[2]').text
print (duracion_vuelo)

23 h 5 min


Para desplegar esa información de las esclas, debemos clickear en link para que se habilite el modal que contiene la informacion. Seleccionémoslo:

In [35]:
link_escalas = vuelo_1.find_element('xpath','//div[@class="sc-fYAFcb kdctDt"]//a')
print (link_escalas)

<selenium.webdriver.remote.webelement.WebElement (session="30b5a596-d633-47be-b92e-51501d732334", element="c0f163fc-4bed-4785-bb6e-d8e0f7649ce0")>


Una vez ubicado el elemento podemos clickear sobre él

In [38]:
link_escalas.click()

Y vemos cómo se despliega la información que estamos buscando. **Notar que cambió el html de la página al hacer click sobre ese botón**

ahora debemos seleccionar los segmentos que contiene el vuelo para calcular la cantidad de escalas, actualmente vemos que las paradas estan contenidas en elementos sections, asi que debemos contabilizar las paradas

In [39]:
paradas= link_escalas.find_elements('xpath','//section[@class="sc-fEVUGC gIelIH"]')
print(paradas)

[<selenium.webdriver.remote.webelement.WebElement (session="30b5a596-d633-47be-b92e-51501d732334", element="7925cac6-dcd6-4095-b814-cb90c2f0b262")>, <selenium.webdriver.remote.webelement.WebElement (session="30b5a596-d633-47be-b92e-51501d732334", element="0c5a1db1-7c65-40db-b24e-e6c591391a78")>]


para calcular las escalas, debemos restar -1 la cantidad total de paradas

In [41]:
escalas=len(paradas)-1
print(escalas)

1


Paso 4: cerrar el navegador

In [ ]:
driver.close()